# 01 — Data Cleaning

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Load Data

In [ ]:
df = pd.read_csv('../data/raw/otodom.csv')
print(f'Shape: {df.shape}')
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

## Missing Values

In [ ]:
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)
missing_df = pd.DataFrame({'count': missing, 'percent': missing_pct})
missing_df[missing_df['count'] > 0].sort_values('percent', ascending=False)

In [ ]:
plt.figure(figsize=(10, 6))
sns.heatmap(df.isnull(), cbar=True, yticklabels=False, cmap='viridis')
plt.title('Missing Values Heatmap')
plt.tight_layout()
plt.show()

## Duplicates

In [ ]:
print(f'Duplicate rows: {df.duplicated().sum()}')
print(f'Duplicate posting_ids: {df["posting_id"].duplicated().sum()}')

In [ ]:
df = df.drop_duplicates(subset='posting_id', keep='first')
print(f'Shape after removing duplicates: {df.shape}')

## Column Types & Fixes

In [ ]:
df.dtypes

In [ ]:
df['price'] = pd.to_numeric(df['price'], errors='coerce')
df['surface'] = pd.to_numeric(df['surface'], errors='coerce')
df['no_of_rooms'] = pd.to_numeric(df['no_of_rooms'], errors='coerce')
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')

## Price per m²

In [ ]:
df['price_per_m2'] = df['price'] / df['surface']
df[['price', 'surface', 'price_per_m2']].describe()

## Location Parsing

In [ ]:
df['location'].head(10)

In [ ]:
location_split = df['location'].str.split(', ', expand=True)
location_split.head(10)

In [ ]:
df['voivodeship'] = location_split.iloc[:, -1]
df['city'] = location_split.iloc[:, -2]
df[['location', 'city', 'voivodeship']].head(10)

## Outliers

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

sns.boxplot(y=df['price'], ax=axes[0])
axes[0].set_title('Price')

sns.boxplot(y=df['surface'], ax=axes[1])
axes[1].set_title('Surface')

sns.boxplot(y=df['price_per_m2'], ax=axes[2])
axes[2].set_title('Price per m²')

plt.tight_layout()
plt.show()

In [ ]:
def remove_outliers_iqr(dataframe, column):
    Q1 = dataframe[column].quantile(0.25)
    Q3 = dataframe[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return dataframe[(dataframe[column] >= lower) & (dataframe[column] <= upper)]

In [ ]:
print(f'Before: {len(df)}')
df_clean = remove_outliers_iqr(df, 'price')
df_clean = remove_outliers_iqr(df_clean, 'surface')
df_clean = remove_outliers_iqr(df_clean, 'price_per_m2')
print(f'After: {len(df_clean)}')

## Drop Unnecessary Columns

In [ ]:
cols_to_drop = ['url', 'posting_id', 'description', 'title', 'timestamp', 'remote_support', 'lighting']
df_clean = df_clean.drop(columns=cols_to_drop, errors='ignore')
df_clean.columns.tolist()

## Final Check

In [ ]:
print(f'Final shape: {df_clean.shape}')
print(f'Missing values:\n{df_clean.isnull().sum()}')
df_clean.head()

## Save Cleaned Data

In [ ]:
df_clean.to_csv('../data/processed/otodom_cleaned.csv', index=False)
print('Saved to data/processed/otodom_cleaned.csv')